2. add customer system prompt and invocation 

In [17]:
import os 
import sys
from dotenv import load_dotenv
from deepagents import create_deep_agent
from langchain_groq import ChatGroq

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

# model = ChatGoogleGenerativeAI(model = "models/gemini-2.5-flash-lite" , api_key=os.getenv("GEMINI_API_KEY"))

model = ChatGroq(model = "qwen/qwen3-32b", api_key=os.getenv("GROQ_API_KEY"))
# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher.
"""

In [18]:

agent = create_deep_agent(
    # tools=[internet_search],
    system_prompt=research_instructions,
    model = model
)

In [19]:
result = agent.invoke({"messages": [{"role": "user", "content": "What is langgraph?"}]})

# Print the agent's response
print(result["messages"][-1].content)

It seems there is no direct information available in the current context about "langgraph" based on the search. The term could refer to various concepts, such as:

1. A graph-based programming or language processing framework
2. A library for visualizing language data as graphs
3. A specific project or tool in NLP (Natural Language Processing)

Would you like me to:
1. Search for additional context about "langgraph"?
2. Provide a general explanation about graph-based language processing concepts?
3. Help you search for specific resources or files related to this term?

Let me know how I can assist further!


In [10]:
print(result)

{'messages': [HumanMessage(content='What is langgraph?', additional_kwargs={}, response_metadata={}, id='dae78bee-9381-41d0-8361-b26a20ab6c5d'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'write_todos', 'arguments': '{"todos": [{"content": "Use internet_search to find a definition and overview of LangGraph.", "status": "in_progress"}, {"status": "pending", "content": "Use internet_search to find key features and concepts of LangGraph (e.g., graphs, nodes, edges, cycles)."}, {"content": "Use internet_search to find examples or common use cases for LangGraph.", "status": "pending"}, {"content": "Synthesize all gathered information into a clear, expert explanation of what LangGraph is.", "status": "pending"}]}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite-preview-09-2025', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b1e47-70fd-7070-a403-b5782f81e478-0', tool_calls=[{'name': 'write_todos', 'args': {'

3. add memory in the agent

In [30]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from deepagents.middleware import FilesystemMiddleware
from deepagents.backends import CompositeBackend, StateBackend, StoreBackend
from langchain.agents import create_agent  # or deepagents.create_deep_agent

# Long-term memory store (persists across threads)
store = InMemoryStore()
checkpointer = MemorySaver()  # Short-term conversation memory

# Composite backend: /memories/ → persistent store, others → ephemeral state
def backend_factory(config):
    return CompositeBackend(
        default=StateBackend(config),
        routes={"/memories/": StoreBackend(store, config)}
    )

agent = create_deep_agent(  # or create_agent with middleware
    model=model,
    checkpointer=checkpointer # Enables short-term memory
)

# Use with thread_id for short-term memory persistence
config = {"configurable": {"thread_id": "user_123"}}
result1 = agent.invoke({"messages": "hey my name is kshitij"}, config)


In [31]:
result1['messages'][-1].content

'Hello Kshitij! 😊 Nice to meet you. How can I assist you today?'

In [32]:

# Later session - short-term via checkpointer, long-term via store
result2 = agent.invoke({"messages": "What's my name ?"}, config)
# Agent recalls "Bob" from short-term memory

In [34]:
result2

{'messages': [HumanMessage(content='hey my name is kshitij', additional_kwargs={}, response_metadata={}, id='92795f06-0cf2-4db9-9815-4aee91aecee0'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user introduced themselves as Kshitij. I need to respond appropriately. Since it\'s a greeting, maybe use the greeting-responder subagent. Let me check the tools available.\n\nThe task tool can launch a subagent. The description says to use the task tool when the user\'s request is complex, but here it\'s just a greeting. However, the example shows using the task tool for greetings with a friendly joke. So I should use the task tool with the greeting-responder subagent type. \n\nWait, the parameters for the task tool require a description and subagent_type. The subagent_type in the example is "greeting-responder". The user\'s message is straightforward, so I should call the task tool with that subagent. The description should be something like "Respond to user greet

In [33]:
result2['messages'][-1].content

'Your name is Kshitij. 😊 You mentioned it earlier! How can I assist you today, Kshitij?'

In [35]:
result2 = agent.invoke({"messages": "Save my name?"}, config)
result2['messages'][-1].content


"Your name, **Kshitij**, is already saved in the file `/user_name.txt`! 😊 Let me know if you'd like to verify it again or need anything else."

In [36]:
result2

{'messages': [HumanMessage(content='hey my name is kshitij', additional_kwargs={}, response_metadata={}, id='92795f06-0cf2-4db9-9815-4aee91aecee0'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user introduced themselves as Kshitij. I need to respond appropriately. Since it\'s a greeting, maybe use the greeting-responder subagent. Let me check the tools available.\n\nThe task tool can launch a subagent. The description says to use the task tool when the user\'s request is complex, but here it\'s just a greeting. However, the example shows using the task tool for greetings with a friendly joke. So I should use the task tool with the greeting-responder subagent type. \n\nWait, the parameters for the task tool require a description and subagent_type. The subagent_type in the example is "greeting-responder". The user\'s message is straightforward, so I should call the task tool with that subagent. The description should be something like "Respond to user greet

In [21]:
import uuid

# Thread 1: Write to long-term memory
config1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
result = agent.invoke({
    "messages": [{"role": "user", "content": "hey, why the sky is blue?"}]
}, config=config1)

In [23]:
result['messages'][-1].content

"The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's a simplified explanation:\n\n1. **Sunlight and Wavelengths**: Sunlight is white light composed of various colors (wavelengths). Blue light has a shorter wavelength (~400-450 nm) compared to red light (~620-750 nm).\n\n2. **Scattering in the Atmosphere**: Earth's atmosphere is filled with molecules of nitrogen and oxygen. When sunlight enters the atmosphere, these molecules scatter the shorter blue wavelengths more efficiently than the longer red wavelengths (because the scattering intensity is inversely proportional to the fourth power of the wavelength, $ \\lambda^{-4} $).\n\n3. **Why Blue, Not Violet?**: While violet light is scattered even more than blue, our eyes are less sensitive to violet, and the sun emits less violet light. The combination of these factors makes the sky appear blue to us.\n\n4. **Day vs. Sunset**: During sunrise or sunset, sunlight travels through more atmosphere, scattering out t

In [24]:
result = agent.invoke({
    "messages": [{"role": "user", "content": "what was my last question?"}]
}, config=config1)
result['messages'][-1].content


"I don't have access to conversation history or memory of previous interactions. If you're looking for your last question in a file or document, I can help you search for it using tools like `grep` or `read_file` if you provide file paths or content details. Let me know how I can assist!"

4. create a subagent - sales agent 
Sales Agent:
- Identifies the RFPs which are due for submission in the next 3 months.
- Scans identified web URLs to summarize RFPs with their due dates.
- Identifies 1 RFP to be selected for response and sends this to the Main
agent.


Lets use the Langraph as orchestration layer for this. Connecting the different agents using Langgraph Node as we want every agent to do some specific task.

Lets create a Agent State which will be persistent across the agents. This will help us to maintain the context across the agents.

In [5]:
# load the RFP document form the artifact folder

from langchain_docling import DoclingLoader
FILE_PATH = ["https://drive.google.com/uc?export=download&id=1pVqPmP_dQxXaOVrJA48Zzg8NP5oem-i_"]
loader = DoclingLoader(file_path=FILE_PATH)


In [ ]:
## creating the 

In [ ]:
documents = loader.load()
documents[:2]

2025-12-16 14:23:13,868 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-16 14:23:17,125 - INFO - Going to convert document batch...
2025-12-16 14:23:17,129 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-16 14:23:17,353 - INFO - Loading plugin 'docling_defaults'
2025-12-16 14:23:17,454 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-12-16 14:23:17,457 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-16 14:23:17,502 - INFO - Loading plugin 'docling_defaults'
2025-12-16 14:23:17,701 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-12-16 14:23:17,703 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-16 14:23:18,436 - INFO - rapidocr cannot be used because onnxruntime i

[Document(metadata={'source': 'https://drive.google.com/uc?export=download&id=1pVqPmP_dQxXaOVrJA48Zzg8NP5oem-i_', 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/4', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 72.0, 't': 575.385, 'r': 518.766, 'b': 514.036, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 380]}]}, {'self_ref': '#/texts/5', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 72.0, 't': 484.545, 'r': 581.987, 'b': 423.65, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 428]}]}, {'self_ref': '#/texts/6', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 72.0, 't': 402.825, 'r': 515.736, 'b': 367.25, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 213]}]}], 'headings': ['R

In [10]:
documents[0].metadata

{'source': 'https://drive.google.com/uc?export=download&id=1pVqPmP_dQxXaOVrJA48Zzg8NP5oem-i_',
 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta',
  'version': '1.0.0',
  'doc_items': [{'self_ref': '#/texts/4',
    'parent': {'$ref': '#/body'},
    'children': [],
    'content_layer': 'body',
    'label': 'text',
    'prov': [{'page_no': 1,
      'bbox': {'l': 72.0,
       't': 575.385,
       'r': 518.766,
       'b': 514.036,
       'coord_origin': 'BOTTOMLEFT'},
      'charspan': [0, 380]}]},
   {'self_ref': '#/texts/5',
    'parent': {'$ref': '#/body'},
    'children': [],
    'content_layer': 'body',
    'label': 'text',
    'prov': [{'page_no': 1,
      'bbox': {'l': 72.0,
       't': 484.545,
       'r': 581.987,
       'b': 423.65,
       'coord_origin': 'BOTTOMLEFT'},
      'charspan': [0, 428]}]},
   {'self_ref': '#/texts/6',
    'parent': {'$ref': '#/body'},
    'children': [],
    'content_layer': 'body',
    'label': 'text',
    'prov': [{'page_no': 1,
 

In [15]:
print(documents[0].page_content)

REQUEST FOR PROPOSAL ANNUAL CONTRACT FOR ENGINEERING SERVICES COUNTY OF CAROLINE, VIRGINIA
Your firm is invited to submit a proposal to provide Engineering Services for the County of Caroline, Public Works, and Utilities in accordance with the enclosed specifications. The submittal, consisting of the original proposal and five  (5) additional  copies  marked, "Proposal for Engineering Services," will be received no later than 2:00 p.m., Friday, October  23, 2020, by:
IN PERSON OR SPECIAL COURIER                       U.S. POSTAL SERVICE County of Caroline                                                     County of Caroline Department of Public Works/Utilities                         Department of Public Works/Utilities 12613 Mill Creek Road                            OR            P O Box 424 Ruther Glen, Virginia 22546                                      Bowling Green, Virginia 22427
This  R.F.P.  and  any  addenda  are  available  on  the  County  of  Caroline  Purchasing website 

In [ ]:
from docling.document_converter import DocumentConverter

source = "https://drive.google.com/uc?export=download&id=1pVqPmP_dQxXaOVrJA48Zzg8NP5oem-i_"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"

Lets create a tools so the sales agent can call this to get the RFPs due in next 3 months.


In [33]:
from langchain_core.tools import tool
from pathlib import Path
import hashlib
from docling.document_converter import DocumentConverter

@tool 
def get_pending_rfps() -> str:
    """Returns a list of RFPs due in next 3 months from the document repository."""
    # For demonstration, returning a static list

    pending_rfps = [{"source":"https://drive.google.com/uc?export=download&id=1pVqPmP_dQxXaOVrJA48Zzg8NP5oem-i_", "due_date":"20-12-2025", "status": "open"}]

    return str(pending_rfps)


@tool
def docling_convert(source: str) -> str:
    """Convert PDF/document to markdown and SAVE to filesystem.
    
    Args:
        source: URL or file path
        
    Returns: FILENAME where markdown is saved (use read_file to access content)
    """
    try:
        converter = DocumentConverter()
        result = converter.convert(source)
        markdown = result.document.export_to_markdown()
        print(markdown)
        ## save the markdown to file 
        return markdown
        # # Create unique filename in agent's workspace
        # filename = f"/research/doc_{Path(source).stem}.md"
        
        # # Agent will call write_file(filename, markdown) next
        # # Return ONLY filename to keep context tiny
        # return f"SAVED: {filename}\n(Use read_file('{filename}') to access full content)"
        
    except Exception as e:
        return f"ERROR: {str(e)}"
    # try:
    #     converter = DocumentConverter()
    #     result = converter.convert(source)
    #     markdown = result.document.export_to_markdown()
    #     doc_id = hashlib.md5(source.encode()).hexdigest()[:8]
    #     filename = f"/research/rfp_{doc_id}.md"
    #     return f"✅ SAVED: {filename}\n📏 {len(markdown)} chars\n➡️ read_file('{filename}')"
    # except Exception as e:
    #     return f"❌ ERROR: {str(e)}"

In [34]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from deepagents.backends import CompositeBackend, StateBackend, StoreBackend
from datetime import datetime
from deepagents.middleware import FilesystemMiddleware

from deepagents import create_deep_agent

todays_date = datetime.now().date()

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]



# Long-term memory store (persists across threads)
store = InMemoryStore()
checkpointer = MemorySaver()  # Short-term conversation memory
config = {"configurable": {"thread_id": "sales_thread_7"}}

# Composite backend: /memories/ → persistent store, others → ephemeral state
def backend_factory(config):
    return CompositeBackend(
        default=StateBackend(config),
        routes={"/memories/": StoreBackend(store, config)}
    )

# Use with thread_id for short-term memory persistence
def sales_agent_node(state: AgentState): 
    sales_agent = create_deep_agent(
        model = model,
        system_prompt="""You are a sales expert analyzing RFPs. 
        1. Scan the RFPs using `get_pending_rfps` tool
        2. Choose the most relevant RFP which is due in next 3 months, todays date is {todays_date}
        3. Convert the RFP document to markdown using `docling_convert` tool, it will return a FILENAME
        4. Read the markdown
        5. Use `write_todos` to plan analysis  
        6. Write analysis to /memories/sales_analysis.txt""".format(todays_date=todays_date),
        checkpointer = checkpointer,
        store = store,
        tools = [get_pending_rfps, docling_convert]

    )
    response = sales_agent.invoke({
        "messages": state['messages'][-1], "configurable": config
    })
    # print("Sales Agent Response:", response['messages'][-1].content)
    return response


In [35]:
## lets run the sales agent node
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
graph = StateGraph(AgentState) 
graph.add_node(sales_agent_node, name="sales_agent_node")
graph.add_edge(START, "sales_agent_node")
graph.add_edge("sales_agent_node", END)
app = graph.compile(checkpointer=checkpointer, store=store)

## display the graph 
# display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI


load_dotenv()
# model = ChatGoogleGenerativeAI(model = "models/gemini-2.5-flash-lite" , api_key=os.getenv("GEMINI_API_KEY"))
# model = ChatGroq(model = "qwen/qwen3-32b", api_key=os.getenv("GROQ_API_KEY"))

In [36]:
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="nvidia/nemotron-3-nano-30b-a3b:free"
)

# # Example usage
# response = llm.invoke("What NFL team won the Super Bowl in the year Justin Bieber was born?")
# print(response.content)


In [37]:
result = app.invoke({
    "messages": [{"role": "user", "content": "Scan the RFPs and provide analysis."}]
}, config=config)
result['messages'][-1].content


2025-12-16 18:06:27,506 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:06:31,326 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:06:41,338 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-16 18:06:42,550 - INFO - Going to convert document batch...
2025-12-16 18:06:42,554 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-16 18:06:44,877 - INFO - Loading plugin 'docling_defaults'
2025-12-16 18:06:44,890 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-12-16 18:06:44,891 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-16 18:06:45,004 - INFO - Loading plugin 'docling_defaults'
2025-12-16 18:06:45,050 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with a

<!-- image -->

## COMMONWEALTH OF VIRGINIA COUNTY OF CAROLINE

## RFP #2020-05-21

## September 16, 2020

## REQUEST FOR PROPOSAL ANNUAL CONTRACT FOR ENGINEERING SERVICES COUNTY OF CAROLINE, VIRGINIA

Your firm is invited to submit a proposal to provide Engineering Services for the County of Caroline, Public Works, and Utilities in accordance with the enclosed specifications. The submittal, consisting of the original proposal and five  (5) additional  copies  marked, "Proposal for Engineering Services," will be received no later than 2:00 p.m., Friday, October  23, 2020, by:

IN PERSON OR SPECIAL COURIER                       U.S. POSTAL SERVICE County of Caroline                                                     County of Caroline Department of Public Works/Utilities                         Department of Public Works/Utilities 12613 Mill Creek Road                            OR            P O Box 424 Ruther Glen, Virginia 22546                                      Bowling Green, Vi

2025-12-16 18:08:15,175 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:08:28,285 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:08:32,928 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:08:44,674 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:09:16,928 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:09:22,693 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:09:26,368 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:10:01,656 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-16 18:10:06,329 - INFO - HTTP Request: POST https://openrouter.a

'**RFP Analysis Summary**\n\n## RFP Metadata\n- **Title:** RFP #2020-05-21 – Annual Contract for Engineering Services – County of Caroline, Virginia  \n- **Issue Date:** September\u202f16,\u202f2020  \n- **Proposal Due Date:** October\u202f23,\u202f2020 (2:00\u202fp.m.)  \n- **Status:** Open  \n- **Due Date (within 3\u202fmonths):** 20‑12‑2025 (4\u202fdays from today, 2025‑12‑16)\n\n## Scope of Services\n- **Professional Engineering Services** for potable water, sanitary sewer, and solid‑waste systems.  \n- Encompasses **design, planning, environmental engineering, construction administration, project cost estimation, bidding, and construction inspection**.  \n- **Work Phases:**  \n  1. **Study & Preliminary Design** – task definition, scope negotiation, preliminary design documents, cost estimate.  \n  2. **Final Design** – final construction plans & specs, revised cost estimate.  \n  3. **Bidding/Negotiating** – assistance with bid acquisition, contractor evaluation.  \n  4. **Constr

In [39]:
print(result['messages'][-1].content )

**RFP Analysis Summary**

## RFP Metadata
- **Title:** RFP #2020-05-21 – Annual Contract for Engineering Services – County of Caroline, Virginia  
- **Issue Date:** September 16, 2020  
- **Proposal Due Date:** October 23, 2020 (2:00 p.m.)  
- **Status:** Open  
- **Due Date (within 3 months):** 20‑12‑2025 (4 days from today, 2025‑12‑16)

## Scope of Services
- **Professional Engineering Services** for potable water, sanitary sewer, and solid‑waste systems.  
- Encompasses **design, planning, environmental engineering, construction administration, project cost estimation, bidding, and construction inspection**.  
- **Work Phases:**  
  1. **Study & Preliminary Design** – task definition, scope negotiation, preliminary design documents, cost estimate.  
  2. **Final Design** – final construction plans & specs, revised cost estimate.  
  3. **Bidding/Negotiating** – assistance with bid acquisition, contractor evaluation.  
  4. **Construction Administration** – site inspections, quality 

In [44]:
result['messages'][-1].content

'Scan the RFPs and provide analysis.'